In [0]:
#imports:
import pandas as pd
import numpy as np
import math
import os
import json 

import xgboost

In [2]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score

In [3]:
from google.colab import drive 
drive.mount('/mntDrive') 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


In [0]:
PREP_DATA_PATH = "/mntDrive/My Drive/GrBoost/Практика/data/input/prepared_facebook_data.csv"
data=pd.read_csv(PREP_DATA_PATH)

RES_SAVE_DIR = "/mntDrive/My Drive/GrBoost/Практика/data/results/test 3"

In [5]:
!pip install ndjson

In [0]:
import ndjson

In [0]:
files = os.listdir(RES_SAVE_DIR)
for file in files:
  print(file)
  with open (RES_SAVE_DIR+"/"+file, "r") as read_file:
    ans = len(ndjson.load(read_file))
  print("Num of objects: ", ans)

BO_simple_1.json
Num of objects:  729
BO_simple_2.json
Num of objects:  729
BO_simple_3.json
Num of objects:  729
BO_simple_4.json
Num of objects:  723
BO_simple_9.json
Num of objects:  729
BO_simple_10.json
Num of objects:  729
BO_simple_8.json
Num of objects:  729


In [0]:
with open (RES_SAVE_DIR+"/BO_simple_4.json", "r") as read_file:
    part_1 = ndjson.load(read_file)
with open (RES_SAVE_DIR+"/BO_simple_4_1.json", "r") as read_file:
    part_2 = ndjson.load(read_file)
part_1.extend(part_2)
with open(RES_SAVE_DIR+"/BO_simple_4_concat.json", 'w') as f:
    ndjson.dump(part_1, f)

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# get some data
X, y = data.iloc[:,:-1].values, data.iloc[:,-1].values

In [0]:
dtrain = xgboost.DMatrix(X, label=y)

In [0]:
def my_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

In [0]:
def bayes_opt_tune_xgb(max_depth=3.0, min_child_weight=1.0, alpha=0.0, lam=1.0, subsample=1.0, colsample_bytree=1.0):
  params = {
      "tree_method": "gpu_hist", 
      "gpu_id": 0, 
      "verbosity": 0,
      "max_depth": int(round(max_depth)),
      "min_child_weight": int(round(min_child_weight)),
      "alpha": alpha,
      "lambda":lam,
      "subsample":subsample,
      "colsample_bytree":colsample_bytree
      }
    #Cross validating with the specified parameters in 5 folds
  cv_result = xgboost.cv(params, dtrain, nfold=5, feval=my_r2_score,  maximize=True)
  return cv_result['test-r2-mean'].iloc[-1]

In [0]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [0]:
# Without groups

In [0]:
restarts=int(round((729-10)/10))

In [0]:
restarts

72

In [0]:
for i in range(8,9):
  xgb_bayes_opt = BayesianOptimization(bayes_opt_tune_xgb, {
      "max_depth": (3, 10),
      "min_child_weight": (1, 10),
      "alpha": (0.0, 1.0),
      "lam": (0.0, 1.0),
      "subsample":(0.5, 0.9),
      "colsample_bytree":(0.5, 0.9)
      })
  logger = JSONLogger(path=RES_SAVE_DIR+"/BO_simple_{}.json".format(i))
  xgb_bayes_opt.subscribe(Events.OPTIMIZATION_STEP, logger)
  xgb_bayes_opt.maximize(n_iter=719, n_restarts_optimizer = restarts, init_points=10)


In [0]:
# problem: BO_simple_4.json
from bayes_opt.util import load_logs
new_optimizer = BayesianOptimization(
    f=bayes_opt_tune_xgb,
    pbounds={
      "max_depth": (3, 10),
      "min_child_weight": (1, 10),
      "alpha": (0.0, 1.0),
      "lam": (0.0, 1.0),
      "subsample":(0.5, 0.9),
      "colsample_bytree":(0.5, 0.9)
      }
)
load_logs(new_optimizer, logs=[RES_SAVE_DIR+"/BO_simple_4.json"]);

logger = JSONLogger(path=RES_SAVE_DIR+"/BO_simple_4_1.json")
new_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

new_optimizer.maximize(
    init_points=0,
    n_iter=6,
)


In [0]:
# with groups

In [0]:
#restarts=int(round((243-10)/10))

In [0]:
"""
for i in range(1,3):
  logger = JSONLogger(path=RES_SAVE_DIR+"/BO_groups_without_stop_{}.json".format(i))
  # first group
  params = {
      "max_depth": (3, 10),
      "min_child_weight": (1, 10)
      }

  xgb_bayes_opt = BayesianOptimization(bayes_opt_tune_xgb, pbounds = params)
  xgb_bayes_opt.subscribe(Events.OPTIMIZATION_STEP, logger)
  xgb_bayes_opt.maximize(n_iter=233, n_restarts_optimizer=restarts, init_points=10)

  best_params = xgb_bayes_opt.max['params']
  # second group
  params["max_depth"] = (int(round(best_params["max_depth"])), int(round(best_params["max_depth"])))
  params["min_child_weight"] = (int(round(best_params["min_child_weight"])), int(round(best_params["min_child_weight"])))
  params.update({"alpha": (0.0, 1.0), "lam": (0.0, 1.0)})

  xgb_bayes_opt = BayesianOptimization(bayes_opt_tune_xgb, pbounds = params)
  xgb_bayes_opt.subscribe(Events.OPTIMIZATION_STEP, logger)
  xgb_bayes_opt.maximize(n_iter=233, n_restarts_optimizer=restarts, init_points=10)
  best_params = xgb_bayes_opt.max['params']
  
  # third group
  params["alpha"] = (best_params["alpha"], best_params["alpha"])
  params["lam"] = (best_params["lam"], best_params["lam"])
  params.update({"subsample":(0.5, 0.9), "colsample_bytree":(0.5, 0.9)})

  xgb_bayes_opt = BayesianOptimization(bayes_opt_tune_xgb, pbounds = params)
  xgb_bayes_opt.subscribe(Events.OPTIMIZATION_STEP, logger)
  xgb_bayes_opt.maximize(n_iter=233, n_restarts_optimizer=restarts, init_points=10)
"""

'\nfor i in range(1,3):\n  logger = JSONLogger(path=RES_SAVE_DIR+"/BO_groups_without_stop_{}.json".format(i))\n  # first group\n  params = {\n      "max_depth": (3, 10),\n      "min_child_weight": (1, 10)\n      }\n\n  xgb_bayes_opt = BayesianOptimization(bayes_opt_tune_xgb, pbounds = params)\n  xgb_bayes_opt.subscribe(Events.OPTIMIZATION_STEP, logger)\n  xgb_bayes_opt.maximize(n_iter=233, n_restarts_optimizer=restarts, init_points=10)\n\n  best_params = xgb_bayes_opt.max[\'params\']\n  # second group\n  params["max_depth"] = (int(round(best_params["max_depth"])), int(round(best_params["max_depth"])))\n  params["min_child_weight"] = (int(round(best_params["min_child_weight"])), int(round(best_params["min_child_weight"])))\n  params.update({"alpha": (0.0, 1.0), "lam": (0.0, 1.0)})\n\n  xgb_bayes_opt = BayesianOptimization(bayes_opt_tune_xgb, pbounds = params)\n  xgb_bayes_opt.subscribe(Events.OPTIMIZATION_STEP, logger)\n  xgb_bayes_opt.maximize(n_iter=233, n_restarts_optimizer=restarts,